In [1]:
import os 
import re
import sys

import jieba
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
data_in = "../data/baidu_95.csv"

In [4]:
df = pd.read_csv(data_in, header=None).rename(columns={0:"label", 1:"content"})

In [5]:
df.head()

,label,content
0,高中 生物 分子与细胞 组成细胞的化学元素 组成细胞的化合物,菠菜从土壤中吸收的氮元素可以用来合成（）A.淀粉和纤维素B.葡萄糖和DNAC.核酸和蛋白质D...
1,高中 生物 稳态与环境 神经调节和体液调节的比较,下列有关生物体内信息传递的叙述，正确的是（）A下丘脑分泌的促甲状腺激素释放激素，可作用于甲状...
2,高中 生物 生物技术实践 生物工程技术,从自然菌样筛选较理想生产菌种的一般步骤是：采集菌样→富集培养→纯种分离→性能测定．1.不同微...
3,高中 生物 生物技术实践 生物技术在其他方面的应用 器官移植 复等位基因 胚胎移植 基因工程...,目前，精子载体法逐渐成为具有诱惑力的制备转基因动物方法之一，该方法以精子作为外源基因的载体，...
4,高中 地理 宇宙中的地球 地球运动的地理意义,某人想乘普通飞机在一年中连续过两次生日，你认为应穿越（）A赤道B两级C本初子午线D国际日期变更线


## 1. 数据处理

### 1.1 加载停用词

In [7]:
from typing import List
def load_stop_words(stop_word_path: str) -> List:
    file = open(stop_word_path, "r", encoding="utf-8")
    
    stop_words = file.readlines()
    
    stop_words = [stop_word.strip() for stop_word in stop_words]
    
    file.close()

    return stop_words

### 1.2 过滤字符

In [8]:
def clean_sentence(line: str) -> List:
    line = re.sub("[a-zA-Z0-9]|[\s+\-\|\!\/\[\]\{\}_,.$%^*(+\"\')]+|[:：+——()?【】《》“”！，。？、~@#￥%……&*（）]+|题目", '',line)
    
    words = jieba.cut(line, cut_all=False)
    
    return words

In [9]:
stopwords_path = "../data/stopwords/哈工大停用词表.txt"

In [10]:
stop_words = load_stop_words(stopwords_path)

def sentence_proc(sentence: str) -> str:
    words = clean_sentence(sentence)
    
    words = [word for word in words if word not in stop_words]
    
    return " ".join(words)

In [11]:
def proc(df):
    df["content"] = df["content"].apply(sentence_proc)
    
    return df

In [12]:
df.describe()

,label,content
count,22576,22576
unique,673,18419
top,高中 地理 宇宙中的地球 地球运动的地理意义,下列四幅图中，能正确表示地球自转方向的是（）ABCD
freq,1533,12


In [13]:
%%time 
data_df = proc(df)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.535 seconds.
Prefix dict has been built successfully.


CPU times: user 17.7 s, sys: 215 ms, total: 17.9 s
Wall time: 18 s


In [14]:
data_df.to_csv("../data/95_baidu_seg.csv")

In [15]:
data_df.head()

,label,content
0,高中 生物 分子与细胞 组成细胞的化学元素 组成细胞的化合物,菠菜 土壤 中 吸收 氮 元素 用来 合成 淀粉 纤维素 葡萄糖 核酸 蛋白质 麦芽糖 脂肪酸
1,高中 生物 稳态与环境 神经调节和体液调节的比较,下列 生物体 内 信息 传递 叙述 正确 下丘脑 分泌 促 甲状腺 激素 释放 激素 作用 ...
2,高中 生物 生物技术实践 生物工程技术,自然 菌样 筛选 理想 生产 菌种 步骤 采集 菌样 富集 培养 纯种 分离 性能 测定 不...
3,高中 生物 生物技术实践 生物技术在其他方面的应用 器官移植 复等位基因 胚胎移植 基因工程...,目前 精子 载体 法 逐渐 成为 具有 诱惑力 制备 转基因 动物 方法 方法 精子 外源 ...
4,高中 地理 宇宙中的地球 地球运动的地理意义,某人 想 普通 飞机 一年 中 连续 两次 生日 认为 应 穿越 赤道 两级 本初子午线 国...


## 2. Vocab

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import MultiLabelBinarizer

In [18]:
vocab_size = 30000
padding_size = 200

## 2.1 ids

In [19]:
text_preprocesser = Tokenizer(num_words=vocab_size, oov_token="<UNK>")
text_preprocesser.fit_on_texts(data_df["content"])

In [20]:
x = text_preprocesser.texts_to_sequences(data_df["content"])

In [21]:
x[0]

[3396, 402, 2, 495, 2481, 605, 1768, 107, 1002, 682, 359, 470, 55, 2290, 3863]

In [22]:
word_dict = text_preprocesser.word_index

In [25]:
dir(text_preprocesser)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_keras_api_names',
 '_keras_api_names_v1',
 'char_level',
 'document_count',
 'filters',
 'fit_on_sequences',
 'fit_on_texts',
 'get_config',
 'index_docs',
 'index_word',
 'lower',
 'num_words',
 'oov_token',
 'sequences_to_matrix',
 'sequences_to_texts',
 'sequences_to_texts_generator',
 'split',
 'texts_to_matrix',
 'texts_to_sequences',
 'texts_to_sequences_generator',
 'to_json',
 'word_counts',
 'word_docs',
 'word_index']

In [26]:
text_preprocesser.num_words

30000

### 2.3 padding

In [27]:
x = pad_sequences(x, maxlen=padding_size, padding="post", truncating="post")

In [30]:
x[0].shape

(200,)

## 3. 分类

多分类与多标签分类label的处理方式不同

## 3.1 数据处理

In [31]:
df["subject"] = df["label"].apply(lambda x: x.split()[1])

In [32]:
df.head()

,label,content,subject
0,高中 生物 分子与细胞 组成细胞的化学元素 组成细胞的化合物,菠菜 土壤 中 吸收 氮 元素 用来 合成 淀粉 纤维素 葡萄糖 核酸 蛋白质 麦芽糖 脂肪酸,生物
1,高中 生物 稳态与环境 神经调节和体液调节的比较,下列 生物体 内 信息 传递 叙述 正确 下丘脑 分泌 促 甲状腺 激素 释放 激素 作用 ...,生物
2,高中 生物 生物技术实践 生物工程技术,自然 菌样 筛选 理想 生产 菌种 步骤 采集 菌样 富集 培养 纯种 分离 性能 测定 不...,生物
3,高中 生物 生物技术实践 生物技术在其他方面的应用 器官移植 复等位基因 胚胎移植 基因工程...,目前 精子 载体 法 逐渐 成为 具有 诱惑力 制备 转基因 动物 方法 方法 精子 外源 ...,生物
4,高中 地理 宇宙中的地球 地球运动的地理意义,某人 想 普通 飞机 一年 中 连续 两次 生日 认为 应 穿越 赤道 两级 本初子午线 国...,地理


In [33]:
df.describe()

,label,content,subject
count,22576,22576,22576
unique,673,18114,4
top,高中 地理 宇宙中的地球 地球运动的地理意义,下列 四幅 图中能 正确 表示 地球 自转 方向,生物
freq,1533,21,13285


In [34]:
set(df["subject"])

{'历史', '地理', '政治', '生物'}

### 3.2 编码

In [35]:
from sklearn.preprocessing import LabelBinarizer

In [36]:
lb = LabelBinarizer()

In [37]:
lb.fit(df["subject"])

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [38]:
y = lb.transform(df["subject"])

In [39]:
y

array([[0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       ...,
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0]])

### 4.3 数据集划分

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [42]:
print('len X_train: ', len(X_train))
print('len X_test: ', len(X_test))
print('len y_train: ', len(y_train))
print('len y_test: ', len(y_test))

len X_train:  18060
len X_test:  4516
len y_train:  18060
len y_test:  4516


## 4. TextCNN

### 4.1 定义 F1

In [17]:
import tensorflow.keras.backend as K

In [18]:
def micro_f1(y_true, y_pred):
    """
    F1 metric.
    
    Computes the micro_f1 and macro_f1, 
    metrics for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=0)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=0)
    
    """
    Micro_F1 metric.
    """
    precision = K.sum(true_positives) / (K.sum(predicted_positives) + K.epsilon())
    recall = K.sum(true_positives) / (K.sum(possible_positives) + K.epsilon())
    micro_f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    
    return micro_f1

In [19]:
def macro_f1(y_true, y_pred):
    """
    F1 metric.

    Computes the micro_f1 and macro_f1,
    metrics for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=0)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=0)

    """
    Macro_F1 metric.
    """
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    macro_f1 = K.mean(2 * precision * recall / (precision + recall + K.epsilon()))

    return macro_f1

In [47]:
import logging 

from tensorflow.keras import Input
from tensorflow.keras.layers import Conv1D, MaxPool1D, Dense, Flatten, concatenate, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from pprint import pprint

In [60]:
def TextCNN(max_sequence_length, max_token_num, embeding_dim, output_dim, model_img_path=None, embedding_matrix=None):
    """
    TextCNN: 
    1.embedding layers, 
    2.convolution layer, 
    3.max-pooling, 
    4.softmax layer. 
    """
    
    x_input = Input(shape=(max_sequence_length,))
    logging.info("x_input.shape: %s" % str(x_input.shape))
    
    if embedding_matrix is None:
        x_emb = Embedding(input_dim=max_token_num, output_dim=embeding_dim, input_length=max_sequence_length)(x_input)
    else:
        x_emb = Embedding(input_dim=max_token_num, output_dim=embeding_dim, input_length=max_sequence_length, 
                          weights=[embedding_matrix], trainable=True)(x_input)
        
    pool_output = []
    kernel_size = [2, 3, 4]
    for kernel_size in kernel_size:
        c = Conv1D(filters=2, kernel_size=kernel_size, strides=1)(x_emb)
        p = MaxPool1D(pool_size=int(c.shape[1]))(c)
        pool_output.append(p)
        
        logging.info("kernel_size: %s \t c.shape: %s \t p.shape: %s" % (kernel_size, str(c.shape), str(p.shape)))
        
    pool_output = concatenate([p for p in pool_output])
    logging.info("pool_output.shape: %s" % str(pool_output.shape))
    
    x_flatten = Flatten()(pool_output)
    y = Dense(output_dim, activation="softmax")(x_flatten)
    
    logging.info("y.shape: %s \n" % str(y.shape))
    
    model = Model([x_input], outputs=[y])
    
    if model_img_path:
        plot_model(model, to_file=model_img_path, show_shapes=True, show_layer_names=False)
    model.summary()
    
    return model


### 4.2 参数设置

In [61]:
feature_size=padding_size
embed_size=300
num_classes=len(y[0])
filter_sizes='3,4,5'
dropout_rate=0.5
regularizers_lambda=0.01
learning_rate=0.01
batch_size=512
epochs=5

### 4.3 训练

In [62]:
model = TextCNN(max_sequence_length=feature_size, max_token_num=vocab_size, embeding_dim=embed_size, output_dim=num_classes)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 200, 300)     9000000     input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 199, 2)       1202        embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 198, 2)       1802        embedding_2[0][0]                
______________________________________________________________________________________________

In [64]:
model.compile(tf.optimizers.Adam(learning_rate=learning_rate), 
              loss='categorical_crossentropy',
              metrics=[micro_f1, macro_f1])

In [65]:
print("Train ...")
early_stopping = EarlyStopping(monitor="val_micro_f1", patience=10, mode="max")

history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          workers=32,
          use_multiprocessing=True,
          callbacks=[early_stopping],
          validation_data=(X_test, y_test))

Train ...
Train on 18060 samples, validate on 4516 samples
Epoch 1/5
18060/18060 [==============================] - 8s 457us/sample - loss: 0.3771 - micro_f1: 0.8058 - macro_f1: 0.6940 - val_loss: 0.0588 - val_micro_f1: 0.9837 - val_macro_f1: 0.9692
Epoch 2/5
18060/18060 [==============================] - 4s 201us/sample - loss: 0.0207 - micro_f1: 0.9943 - macro_f1: 0.9892 - val_loss: 0.0339 - val_micro_f1: 0.9891 - val_macro_f1: 0.9799
Epoch 3/5
18060/18060 [==============================] - 4s 203us/sample - loss: 0.0018 - micro_f1: 0.9996 - macro_f1: 0.9993 - val_loss: 0.0347 - val_micro_f1: 0.9888 - val_macro_f1: 0.9788
Epoch 4/5
18060/18060 [==============================] - 4s 200us/sample - loss: 3.7348e-04 - micro_f1: 0.9999 - macro_f1: 0.9999 - val_loss: 0.0342 - val_micro_f1: 0.9900 - val_macro_f1: 0.9811
Epoch 5/5
18060/18060 [==============================] - 4s 204us/sample - loss: 1.3194e-04 - micro_f1: 1.0000 - macro_f1: 1.0000 - val_loss: 0.0347 - val_micro_f1: 0.9894 -

## 5. 评估

In [66]:
from sklearn.metrics import classification_report, confusion_matrix

In [67]:
y_pred = model.predict(X_test)

In [68]:
y_pred

array([[7.4185596e-07, 9.9999833e-01, 1.5784484e-07, 7.9361536e-07],
       [4.9243496e-05, 2.5088487e-13, 9.9995077e-01, 5.4133181e-15],
       [9.1258023e-08, 3.0592282e-06, 1.3324892e-08, 9.9999690e-01],
       ...,
       [2.2144951e-10, 4.0530301e-07, 7.5984457e-07, 9.9999893e-01],
       [1.2222332e-11, 2.6422731e-07, 1.1754281e-11, 9.9999976e-01],
       [3.8792337e-12, 4.6830598e-07, 1.7006965e-11, 9.9999952e-01]],
      dtype=float32)

In [69]:
y_pred = np.argmax(y_pred, axis=1)

In [70]:
y_pred

array([1, 2, 3, ..., 3, 3, 3])

In [71]:
y_test

array([[0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       ...,
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1]])

In [72]:
y_true = np.argmax(y_test, axis=1)

In [73]:
y_true

array([1, 2, 3, ..., 3, 3, 3])

In [74]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       516
           1       0.98      0.98      0.98       943
           2       0.98      0.96      0.97       393
           3       1.00      1.00      1.00      2664

    accuracy                           0.99      4516
   macro avg       0.98      0.98      0.98      4516
weighted avg       0.99      0.99      0.99      4516



In [75]:
confusion_matrix(y_true,y_pred)

array([[ 505,    7,    4,    0],
       [   7,  927,    5,    4],
       [  12,    4,  376,    1],
       [   1,    4,    0, 2659]])

# 多标签分类

In [2]:
data = pd.read_csv("../data/95_baidu_seg.csv")

In [3]:
data.head()

,Unnamed: 0,label,content
0,0,高中 生物 分子与细胞 组成细胞的化学元素 组成细胞的化合物,菠菜 土壤 中 吸收 氮 元素 用来 合成 淀粉 纤维素 葡萄糖 核酸 蛋白质 麦芽糖 脂肪酸
1,1,高中 生物 稳态与环境 神经调节和体液调节的比较,下列 生物体 内 信息 传递 叙述 正确 下丘脑 分泌 促 甲状腺 激素 释放 激素 作用 ...
2,2,高中 生物 生物技术实践 生物工程技术,自然 菌样 筛选 理想 生产 菌种 步骤 采集 菌样 富集 培养 纯种 分离 性能 测定 不...
3,3,高中 生物 生物技术实践 生物技术在其他方面的应用 器官移植 复等位基因 胚胎移植 基因工程...,目前 精子 载体 法 逐渐 成为 具有 诱惑力 制备 转基因 动物 方法 方法 精子 外源 ...
4,4,高中 地理 宇宙中的地球 地球运动的地理意义,某人 想 普通 飞机 一年 中 连续 两次 生日 认为 应 穿越 赤道 两级 本初子午线 国...


In [4]:
data["label"] = data["label"].apply(lambda x: x.split())

In [5]:
data.head()

,Unnamed: 0,label,content
0,0,"[高中, 生物, 分子与细胞, 组成细胞的化学元素, 组成细胞的化合物]",菠菜 土壤 中 吸收 氮 元素 用来 合成 淀粉 纤维素 葡萄糖 核酸 蛋白质 麦芽糖 脂肪酸
1,1,"[高中, 生物, 稳态与环境, 神经调节和体液调节的比较]",下列 生物体 内 信息 传递 叙述 正确 下丘脑 分泌 促 甲状腺 激素 释放 激素 作用 ...
2,2,"[高中, 生物, 生物技术实践, 生物工程技术]",自然 菌样 筛选 理想 生产 菌种 步骤 采集 菌样 富集 培养 纯种 分离 性能 测定 不...
3,3,"[高中, 生物, 生物技术实践, 生物技术在其他方面的应用, 器官移植, 复等位基因, 胚胎...",目前 精子 载体 法 逐渐 成为 具有 诱惑力 制备 转基因 动物 方法 方法 精子 外源 ...
4,4,"[高中, 地理, 宇宙中的地球, 地球运动的地理意义]",某人 想 普通 飞机 一年 中 连续 两次 生日 认为 应 穿越 赤道 两级 本初子午线 国...


## 多标签编码

In [6]:
from sklearn.preprocessing import MultiLabelBinarizer

In [7]:
mlb = MultiLabelBinarizer()

In [8]:
mlb = mlb.fit(data["label"])

In [9]:
mlb

MultiLabelBinarizer(classes=None, sparse_output=False)

In [10]:
mlb.classes_

array(['“重农抑商”政策', '不完全显性', '与细胞分裂有关的细胞器', '中央官制——三公九卿制', '中心体的结构和功能',
       '人体免疫系统在维持稳态中的作用', '人体水盐平衡调节', '人体的体温调节', '人口与城市', '人口增长与人口问题',
       '人口迁移与人口流动', '人工授精、试管婴儿等生殖技术', '伴性遗传', '体液免疫的概念和过程', '免疫系统的功能',
       '免疫系统的组成', '公民道德与伦理常识', '兴奋在神经元之间的传递', '兴奋在神经纤维上的传导', '内环境的稳态',
       '内质网的结构和功能', '农业区位因素', '减数分裂与有丝分裂的比较', '减数分裂的概念', '分子与细胞',
       '劳动就业与守法经营', '历史', '古代史', '器官移植', '地球与地图', '地球所处的宇宙环境',
       '地球的内部圈层结构及特点', '地球的外部圈层结构及特点', '地球运动的地理意义', '地球运动的基本形式', '地理',
       '垄断组织的出现', '培养基与无菌技术', '基因工程的原理及技术', '基因工程的概念', '基因的分离规律的实质及应用',
       '基因的自由组合规律的实质及应用', '复等位基因', '夏商两代的政治制度', '太阳对地球的影响', '宇宙中的地球',
       '工业区位因素', '拉马克的进化学说', '政治', '文艺的春天', '核糖体的结构和功能', '海峡两岸关系的发展',
       '液泡的结构和功能', '清末民主革命风潮', '溶酶体的结构和功能', '激素调节', '现代史', '现代生物技术专题',
       '生产活动与地域联系', '生命活动离不开细胞', '生态系统的营养结构', '生活中的法律常识', '生物', '生物工程技术',
       '生物性污染', '生物技术在其他方面的应用', '生物技术实践', '生物科学与社会', '皇帝制度',
       '社会主义市场经济的伦理要求', '社会主义是中国人民的历史性选择', '神经调节和体液调节的比较', '科学社会主义常识',
       '稳态与环境', '第三产业的兴起和“新

In [12]:
y = mlb.fit_transform(data["label"])

In [13]:
len(y[0])

95

In [16]:
y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0])

In [20]:
vocab_size = 30000
padding_size = 200